In [26]:
!pip install spotipy
!pip install Flask
!pip install bokeh
#!pip install python-dotenv

In [6]:
from flask import Flask, request

app = Flask(__name__)

@app.route("/")
def index():
    return "Hello, World!"

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 1040, in launch_instance
    app.initialize(argv)
  File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 666, in initialize
    self.init_sockets()
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 307, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 244, in _bind_socket
    return self._try_bind_socket(s, port)
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 220, in _try_bind_socket
    s.bind("tcp://%s:%i" % (sel

SystemExit: 1

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import spotipy
import spotipy.util as util
import requests
import json
import csv

#
#Auth details used to be here :)
#

def authorize_app():
    auth_manager = spotipy.oauth2.SpotifyOAuth(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope=SCOPE
    )

    auth_url = auth_manager.get_authorize_url()
    print('Please visit this URL to authorize the app:', auth_url)

    response = input('Enter the full URL of the page you were redirected to: ')
    auth_response = dict(s.split('=') for s in response.split('&'))

    token_info = auth_manager.get_access_token(auth_response['code'])
    return token_info['access_token']

def get_saved_tracks(access_token):
    headers = {
        'Authorization': 'Bearer ' + access_token
    }

    limit = 50
    offset = 0
    total = None
    saved_tracks = []

    while total is None or offset < total:
        url = f'https://api.spotify.com/v1/me/tracks?limit={limit}&offset={offset}'
        response = requests.get(url, headers=headers)
        data = json.loads(response.text)

        total = data['total']
        offset += limit
        saved_tracks += data['items']

    return saved_tracks

        
def write_to_csv(tracks):
    with open('track_ids.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Track Name', 'Artist', 'Track ID'])
        for track in tracks:
            track_name = track['track']['name']
            track_artist = track['track']['artists'][0]['name']
            track_id = track['track']['id']
            writer.writerow([track_name, track_artist, track_id])

if __name__ == '__main__':
    access_token = authorize_app()
    saved_tracks = get_saved_tracks(access_token)
    write_to_csv(saved_tracks)
    print("Track IDs have been written to 'track_ids.csv' file.")

In [10]:
#
#Auth details used to be here :)
#
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)


In [31]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import ujson
import spotipy
import spotipy.util
import seaborn as sns
from bokeh.plotting import figure, show

p = figure()
# Use p to create Histogram, Scatter, Donut plots
#show(p)

from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh import palettes

In [18]:
# shows audio analysis for the given track

from __future__ import print_function    # (at top of module)
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
import pandas as pd

tracks_df = pd.read_csv('track_ids.csv')


sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

audio_features = {}
for idd in tracks_df['Track ID'].tolist():
    audio_features[idd] = sp.audio_features(idd)[0]


In [39]:
tracks_df['acousticness'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['acousticness'])
tracks_df['speechiness'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['speechiness'])
tracks_df['key'] = tracks_df['Track ID'].apply(lambda idd: str(audio_features[idd]['key']))
tracks_df['liveness'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['liveness'])
tracks_df['instrumentalness'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['instrumentalness'])
tracks_df['energy'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['energy'])
tracks_df['tempo'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['tempo'])
tracks_df['time_signature'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['time_signature'])
tracks_df['loudness'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['loudness'])
tracks_df['danceability'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['danceability'])
tracks_df['mode'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['mode'])
tracks_df['speechiness'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['speechiness'])
tracks_df['time_signature'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['time_signature'])
tracks_df['valence'] = tracks_df['Track ID'].apply(lambda idd: audio_features[idd]['valence'])

output_notebook()

# Create a ColumnDataSource from your tracks DataFrame
source = ColumnDataSource(tracks_df)

# Create a figure for the scatter plots
p1 = figure(title='danceability vs valence', plot_width=800, plot_height=400)
p1.scatter('valence', 'danceability', color='navy', source=source)

p2 = figure(title='loudness vs energy', plot_width=800, plot_height=400)
p2.scatter('energy', 'loudness', color='navy', source=source)

p3 = figure(title='valence vs energy', plot_width=800, plot_height=400)
p3.scatter('energy', 'valence', color='navy', source=source)


# Create a figure object
p = figure(title='Histogram', plot_width=800, plot_height=400)

# Calculate the histogram
hist, edges = np.histogram(source, bins=20)

# Add the histogram glyph to the figure
p4.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color='navy', line_color='black')

# Show the histogram
show(p4)



# Show the scatter plots
show(p1)
show(p2)
show(p3)

Loading BokehJS ...

In [65]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import ujson
import spotipy
import spotipy.util
import seaborn as sns
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh import palettes

# Create a figure object
p = figure(title='Histogram', plot_width=600, plot_height=400)

# Set the bins
bins = 10

# Create histograms for each column
#p.quad(top='loudness', bottom=0, left='loudness[:-1]', right='loudness[1:]', source=source,fill_color='navy', line_color='black', alpha=0.5, legend_label='loudness')
#p.quad(top='hist_column2', bottom=0, left='edges_column2[:-1]', right='edges_column2[1:]', source=source, fill_color='green', line_color='black', alpha=0.5, legend_label='tempo')
#p.quad(top='hist_column3', bottom=0, left='edges_column3[:-1]', right='edges_column3[1:]', source=source,fill_color='orange', line_color='black', alpha=0.5, legend_label='energy')

# Create histogram for 'loudness'
hist_loudness, edges_loudness = np.histogram(tracks_df['loudness'], bins=bins)
p.quad(top=hist_loudness, bottom=0, left=edges_loudness[:-1], right=edges_loudness[1:], fill_color='navy', line_color='black', alpha=0.5, legend_label='Loudness')

# Create histogram for 'tempo'
hist_tempo, edges_tempo = np.histogram(tracks_df['tempo'], bins=bins)
p.quad(top=hist_tempo, bottom=0, left=edges_tempo[:-1], right=edges_tempo[1:], fill_color='green', line_color='black', alpha=0.5, legend_label='Tempo')

# Create histogram for 'danceability'
hist_danceability, edges_danceability = np.histogram(tracks_df['danceability'], bins=bins)
p.quad(top=hist_danceability, bottom=0, left=edges_danceability[:-1], right=edges_danceability[1:],
       fill_color='orange', line_color='black', alpha=0.5, legend_label='Danceability')


# Set the x-axis label
p.xaxis.axis_label = 'Value'

# Set the y-axis label
p.yaxis.axis_label = 'Frequency'

# Show the histogram
show(p)

In [54]:
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        649 non-null    object 
 1   Artist            649 non-null    object 
 2   Track ID          649 non-null    object 
 3   acousticness      649 non-null    float64
 4   speechiness       649 non-null    float64
 5   key               649 non-null    object 
 6   liveness          649 non-null    float64
 7   instrumentalness  649 non-null    float64
 8   energy            649 non-null    float64
 9   tempo             649 non-null    float64
 10  time_signature    649 non-null    int64  
 11  loudness          649 non-null    float64
 12  danceability      649 non-null    float64
 13  mode              649 non-null    int64  
 14  valence           649 non-null    float64
dtypes: float64(9), int64(2), object(4)
memory usage: 76.2+ KB


In [44]:
import sys 
import spotipy
from spotipy.oauth2 import SpotifyOAuth

#
#Auth details used to be here :)
#

token = spotipy.util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri = redirect_uri)

OSError: [Errno 98] Address already in use

In [24]:
# Shows a user's saved tracks (need to be authenticated via oauth)

import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = 'user-library-read'


def show_tracks(results):
    for item in results['items']:
        track = item['track']
        print("%32.32s %s" % (track['artists'][0]['name'], track['name']))


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

results = sp.current_user_saved_tracks()
show_tracks(results)

while results['next']:
    results = sp.next(results)
    show_tracks(results)

SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.

In [ ]:
playlists = spotify.user_playlists('USERNAME_TO_INSPECT')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'],   playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

In [ ]:
print(track_name, artist_name)
print(sp.audio_features(track_uri)[0])

In [ ]:
#https://colinraffel.com/projects/lmd/
#lakh midi data set